In [2]:
import os
import cv2
import numpy as np

# Percorsi delle cartelle
image_dir = "C:\\Users\\Q540900\\Desktop\\A.I. Master\\Werkstudent\\RAVIR Dataset\\train\\training_images"
mask_dir = "C:\\Users\\Q540900\\Desktop\\A.I. Master\\Werkstudent\\RAVIR Dataset\\train\\training_masks"
output_image_dir = "C:\\Users\\Q540900\\Desktop\\A.I. Master\\Werkstudent\\RAVIR Dataset\\train\\patch_images"
output_mask_dir = "C:\\Users\\Q540900\\Desktop\\A.I. Master\\Werkstudent\\RAVIR Dataset\\train\\patch_masks"

# Crea le cartelle di output se non esistono
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)

# Dimensione della patch
patch_size = 256

# Funzione per dividere l'immagine in patch
def generate_patches(image, mask, patch_size):
    patches = []
    mask_patches = []
    h, w = image.shape[:2]
    for i in range(0, h, patch_size):
        for j in range(0, w, patch_size):
            patch = image[i:i+patch_size, j:j+patch_size]
            mask_patch = mask[i:i+patch_size, j:j+patch_size]
            if patch.shape[0] == patch_size and patch.shape[1] == patch_size:
                patches.append(patch)
                mask_patches.append(mask_patch)
    return patches, mask_patches

# Funzione per calcolare la varianza di una patch
def calculate_variance(patch):
    return np.var(patch)

# Itera attraverso le immagini
for filename in os.listdir(image_dir):
    if filename.endswith(".png"):
        image_path = os.path.join(image_dir, filename)
        mask_path = os.path.join(mask_dir, filename)
        
        # Carica immagine e maschera
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        # Genera le patch
        patches, mask_patches = generate_patches(image, mask, patch_size)
        
        # Calcola la varianza e ordina le patch
        variances = [calculate_variance(patch) for patch in mask_patches]
        sorted_indices = np.argsort(variances)[::-1]  # Ordina per varianza decrescente
        
        # Seleziona le patch con maggiore varianza
        top_k = 30  # Numero di patch da selezionare
        for idx in sorted_indices[:top_k]:
            patch = patches[idx]
            mask_patch = mask_patches[idx]
            patch_filename = f"{filename[:-4]}_patch_{idx}.png"
            
            # Salva le patch
            cv2.imwrite(os.path.join(output_image_dir, patch_filename), patch)
            cv2.imwrite(os.path.join(output_mask_dir, patch_filename), mask_patch)

print("Patch generation completed.")


Patch generation completed.
